<a href="https://colab.research.google.com/github/richlysakowski/All_things_medium/blob/main/ReverseGeocoding-DEBUGGED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime


Reading package lists...
Building dependency tree...
Reading state information...
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate
Reading package lists...
Building dependency tree...
Reading state information...
python3-rtree is already the newest version (0.9.4-1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet git://github.com/geopandas/geopandas.git /tmp/pip-req-build-chlb3ckx did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet git://github.com/geopandas/geopandas

* Restart the run time. tqdm does not work with recent python so we downgrade it to get it working

In [6]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 82.7 MB/s eta 0:00:00
time: 9.07 s (started: 2023-01-28 20:33:56 +00:00)


In [4]:
!pip install "tqdm<5.0.0,>=4.48.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 10.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.36.0
    Uninstalling tqdm-4.36.0:
      Successfully uninstalled tqdm-4.36.0


In [7]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.15 s (started: 2023-01-28 20:34:13 +00:00)


<ipython-input-7-9a8a7a5758e8>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


## Reverse Geocoding Single Example

In [8]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "53.480837, -2.244914"
location = locator.reverse(coordinates)

time: 1.01 s (started: 2023-01-28 20:34:20 +00:00)


In [9]:
location.raw

{'place_id': 118137125,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 37139875,
 'lat': '53.4809609',
 'lon': '-2.2450805553523505',
 'display_name': '68, Cross Street, Civic Quarter, City Centre, Manchester, Greater Manchester, England, M2 4JN, United Kingdom',
 'address': {'house_number': '68',
  'road': 'Cross Street',
  'neighbourhood': 'Civic Quarter',
  'suburb': 'City Centre',
  'city': 'Manchester',
  'ISO3166-2-lvl8': 'GB-MAN',
  'state_district': 'Greater Manchester',
  'state': 'England',
  'ISO3166-2-lvl4': 'GB-ENG',
  'postcode': 'M2 4JN',
  'country': 'United Kingdom',
  'country_code': 'gb'},
 'boundingbox': ['53.4808484', '53.4810728', '-2.2451942', '-2.2449675']}

time: 3.54 ms (started: 2023-01-28 20:34:24 +00:00)


In [10]:
print(location.address)

68, Cross Street, Civic Quarter, City Centre, Manchester, Greater Manchester, England, M2 4JN, United Kingdom
time: 594 µs (started: 2023-01-28 20:34:34 +00:00)


In [11]:
location.raw["display_name"]

'68, Cross Street, Civic Quarter, City Centre, Manchester, Greater Manchester, England, M2 4JN, United Kingdom'

time: 3.01 ms (started: 2023-01-28 20:34:39 +00:00)


## Reverse Geocoding with Pandas

In [12]:
url = "https://www.dropbox.com/s/15gisj8hx218rn1/street-pole-sample.csv?dl=1"
cols = ["X", "Y", "POLE_NUM","TYPE","HEIGHT","POLE_DATE","OWNER"]
df = pd.read_csv(url, usecols=cols)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets


time: 2.66 s (started: 2023-01-28 20:34:44 +00:00)


In [13]:
df.shape

(150, 7)

time: 3.3 ms (started: 2023-01-28 20:34:50 +00:00)


In [14]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(df, lat="Y", lon="X",  zoom=15)

time: 895 ms (started: 2023-01-28 20:34:53 +00:00)


In [15]:
df["geom"] =  df["Y"].map(str)  + ',' + df['X'].map(str)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821"
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.9414773141344,-75.166112027818"
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411"
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079"
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191"


time: 13.4 ms (started: 2023-01-28 20:34:59 +00:00)


In [20]:
df.shape

(150, 9)

time: 8.38 ms (started: 2023-01-28 20:37:51 +00:00)


In [16]:
df.geom[0]



'39.9427660880249,-75.17009743393821'

time: 3.59 ms (started: 2023-01-28 20:35:12 +00:00)


In [17]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

time: 4.5 ms (started: 2023-01-28 20:35:15 +00:00)


In [18]:
tqdm.pandas()
df['address'] = df['geom'].progress_apply(rgeocode)

100%|██████████| 150/150 [02:06<00:00,  1.18it/s]

time: 2min 6s (started: 2023-01-28 20:35:17 +00:00)


In [21]:
df.head()


,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom,address
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821","(720, South Chadwick Street, South Philadelphi..."
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.9414773141344,-75.166112027818","(Tindley Temple United Methodist Church, 750, ..."
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411","(623, South 13th Street, Martin Luther King Pl..."
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079","(1518, Naudain Street, Rittenhouse Square, Cen..."
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191","(1726, Catharine Street, Naval Square, South P..."


time: 15.9 ms (started: 2023-01-28 20:38:01 +00:00)


In [22]:
df["address"][0]

Location(720, South Chadwick Street, South Philadelphia, Philadelphia, Philadelphia County, Pennsylvania, 19146, United States, (39.94280445, -75.1702057628465, 0.0))

time: 2.91 ms (started: 2023-01-28 20:38:03 +00:00)


time: 2.33 s (started: 2023-01-28 20:38:03 +00:00)
